In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

# We will import a customized function called client which actually returns an authorized bigquery client object with right credentials
# this will cost us an extra pair of () each time we call the client object which is now called by the function client we define in bq_sa_auth.py 

from bq_sa_auth import client

## **Lecture 2: Analytic (window) functions**

#### Refer to the [tutorial](https://www.kaggle.com/code/alexisbcook/analytic-functions/tutorial) for neat examples and Introduction to the analytic functions like AVG(), MIN/MAX(), RANK(), LAG()/LEAD(), FIRST_VALUE()/LAST_VALUE()

--------------


### Example

#### We'll work with the San Francisco Open Data dataset. We begin by reviewing the first several rows of the bikeshare_trips table.

In [2]:
# Construct a reference to the "san_francisco" dataset
dataset_ref = client().dataset("san_francisco", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client().get_dataset(dataset_ref)

# Construct a reference to the "bikeshare_trips" table
table_ref = dataset_ref.table("bikeshare_trips")

# API request - fetch the table
table = client().get_table(table_ref)

# Preview the first five lines of the table
client().list_rows(table, max_results=5).to_dataframe()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


### Each row of the table corresponds to a different bike trip, and we can use an analytic function to calculate the cumulative number of trips for each date in 2015.

In [3]:
# Query to count the (cumulative) number of trips per day using CTE (Common Table Expression)
num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                      COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                               ) AS cumulative_trips
                      FROM trips_by_day
                  """

# Run the query, and return a pandas DataFrame
num_trips_result = client().query(num_trips_query).result().to_dataframe()
num_trips_result.head()

,trip_date,num_trips,cumulative_trips
0,2015-10-27,1270,300469
1,2015-02-17,1279,43786
2,2015-05-24,333,140775
3,2015-04-15,1359,101380
4,2015-08-15,451,226286


The query uses a common table expression (CTE) to first calculate the daily number of trips. Then, we use SUM() as an aggregate function.

- Since there is no PARTITION BY clause, the entire table is treated as a single partition.
- The ORDER BY clause orders the rows by date, where earlier dates appear first.
- By setting the window frame clause to ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW, we ensure that all rows up to and including the current date are used to calculate the (cumulative) sum. (Note: If you read the documentation, you'll see that this is the default behavior, and so the query would return the same result if we left out this window frame clause.)

The next query tracks the stations where each bike began (in start_station_id) and ended (in end_station_id) the day on October 25, 2015.

In [4]:
# Query to track beginning and ending stations on October 25, 2015, for each bike

b_and_e_stations_query = """
                        SELECT bike_number, TIME(start_date) AS trip_start_time,
                        start_station_id, end_station_id,
                        FIRST_VALUE(start_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS first_station_id,
                               
                        LAST_VALUE(end_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id
                        FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                        WHERE DATE(start_date) = '2015-10-25'
                        """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed =27*10**10)

query_job = client().query(b_and_e_stations_query, job_config=safe_config)

bikes = query_job.to_dataframe()

bikes.head(15)

,bike_number,trip_start_time,start_station_id,end_station_id,first_station_id,last_station_id
0,121,18:13:00,83,83,83,83
1,491,12:43:00,56,73,56,73
2,614,14:13:00,74,60,74,66
3,614,16:39:00,60,66,74,66
4,395,00:24:00,70,72,70,72
5,480,15:56:00,39,67,39,67
6,349,14:37:00,42,75,42,51
7,349,18:01:00,75,51,42,51
8,402,17:18:00,77,65,77,75
9,402,17:42:00,65,75,77,75


In [5]:
bikes[bikes['bike_number'] == 581]

,bike_number,trip_start_time,start_station_id,end_station_id,first_station_id,last_station_id
102,581,12:41:00,77,60,77,73
103,581,12:56:00,60,73,77,73


### **Exercises:**

#### Introduction

Here, we will use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

In [6]:
# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client().dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client().get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client().get_table(table_ref)

# Preview the first five lines of the table
client().list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,d2f7afe2dbe9a7a883bcd2bea8a61b4f1ca9ee50,312a81d2e122cf94378492abd78371da19017e26f1bb7d...,2019-06-25 14:45:00+00:00,2019-06-25 15:00:00+00:00,552,1.40,<NA>,<NA>,<NA>,<NA>,...,0.0,7.75,Cash,City Service,NaN,NaN,None,NaN,NaN,None
1,d2f3833d31b230290e1dbab284da40eb164557ac,4ceed9f280171cebce539cee58acec704cf2f1dca2f89e...,2019-06-03 19:00:00+00:00,2019-06-03 19:00:00+00:00,319,0.96,<NA>,<NA>,<NA>,<NA>,...,0.0,9.50,Credit Card,Chicago Carriage Cab Corp,NaN,NaN,None,NaN,NaN,None
2,d2f82eef85794ca47a8389264c4ec40337d538f8,b71b6ac8ce80a63f080b0b1797320425c67b74af57bdd7...,2019-06-01 21:45:00+00:00,2019-06-01 22:00:00+00:00,608,2.26,<NA>,<NA>,<NA>,<NA>,...,0.0,9.25,Cash,Chicago Carriage Cab Corp,NaN,NaN,None,NaN,NaN,None
3,d2feffe082a67bd9483fc2a6e6c56a7e341fe1eb,63f04d77360d12a3b71ebabb82794a2cc1842a1f5a147c...,2019-06-12 17:30:00+00:00,2019-06-12 17:45:00+00:00,1441,4.23,<NA>,<NA>,<NA>,<NA>,...,0.0,16.50,Cash,Flash Cab,NaN,NaN,None,NaN,NaN,None
4,d3005e16e1cbb94006a6c7219e35ee7aa165678f,4b9b91a8648e40ebafcc374aaf02db473cb8e1953f0c4f...,2019-06-05 16:00:00+00:00,2019-06-05 16:15:00+00:00,531,2.34,<NA>,<NA>,<NA>,<NA>,...,5.0,14.25,Cash,Flash Cab,NaN,NaN,None,NaN,NaN,None


### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [7]:

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      ORDER BY trip_date
                      """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed =27*10**10)

query_job = client().query(avg_num_trips_query, job_config=safe_config)

taxi_trips_avg = query_job.to_dataframe()

taxi_trips_avg.head(15)

,trip_date,avg_num_trips
0,2016-01-01,73067.250000
1,2016-01-02,72801.000000
2,2016-01-03,73287.833333
3,2016-01-04,74260.571429
4,2016-01-05,73767.285714
5,2016-01-06,74708.571429
6,2016-01-07,75520.428571
7,2016-01-08,77646.857143
8,2016-01-09,79842.000000
9,2016-01-10,82083.428571


### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [8]:
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed =27*10**10)

query_job = client().query(trip_number_query, job_config=safe_config)

taxi_trips_by_area = query_job.to_dataframe()

taxi_trips_by_area.head(15)

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,26,2013-10-03 04:30:00+00:00,2013-10-03 04:45:00+00:00,1
1,26,2013-10-03 05:15:00+00:00,2013-10-03 05:15:00+00:00,2
2,37,2013-10-03 01:15:00+00:00,2013-10-03 01:15:00+00:00,1
3,37,2013-10-03 10:00:00+00:00,2013-10-03 10:00:00+00:00,2
4,37,2013-10-03 13:00:00+00:00,2013-10-03 13:15:00+00:00,3
5,37,2013-10-03 14:00:00+00:00,2013-10-03 14:15:00+00:00,4
6,37,2013-10-03 15:15:00+00:00,2013-10-03 16:15:00+00:00,5
7,37,2013-10-03 17:15:00+00:00,2013-10-03 17:30:00+00:00,6
8,37,2013-10-03 19:45:00+00:00,2013-10-03 20:00:00+00:00,7
9,37,2013-10-03 21:15:00+00:00,2013-10-03 21:45:00+00:00,8


### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [9]:
# Added an extra ORDER BY taxi_id to show the same taxi driver at the head of df where the initial prev_break is NaN

break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   ORDER BY taxi_id
                   """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed =27*10**10)

query_job = client().query(break_time_query, job_config=safe_config)

break_time_b_tt = query_job.to_dataframe()

break_time_b_tt.head(5)

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:15:00+00:00,2013-10-03 11:30:00+00:00,<NA>
1,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:30:00+00:00,2013-10-03 11:45:00+00:00,0
2,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:45:00+00:00,2013-10-03 12:00:00+00:00,0
3,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:00:00+00:00,2013-10-03 12:00:00+00:00,0
4,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:15:00+00:00,2013-10-03 12:15:00+00:00,15
